In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import random

In [2]:
class Robot:
    def __init__(self,pos:np.ndarray,antennas:list):
        self.antennas = antennas
        self.position = pos
        self.num_of_antennas = len(antennas)
    def update_pos(self,pos):
        self.position = pos

In [3]:
class Tag:
    def __init__(self,pos:np.ndarray):
        self.position = pos

In [4]:
class Antenna:
    def __init__(self,z:float,G_dBm:float,theta_vector:np.ndarray,phi_vector:np.ndarray):
        self.z = z
        self.G_dBm = G_dBm
        self.theta_vector = theta_vector
        self.phi_vector = phi_vector
    
    def get_gain(self,theta_rad:float,phi_rad:float):
        theta_deg: int = round(theta_rad*180/np.pi)%360 #we don't really need the modulo here, i think but just to be safe
        phi_deg: int = round(phi_rad * 180/np.pi)%360
        return self.theta_vector[theta_deg] + self.phi_vector[phi_deg] + self.G_dBm


In [18]:
#We assume the RFID tag frequency is 866MHz
freq = 866e6
#Speed of light
c = 299792458.0

lambda_signal = c/freq


Pt_dBm = 30

G_R_dBm = 1

#antenna initialization
pattern = pd.read_excel("pattern.xlsx")
gain_theta = pattern["Gain_t(dB)"].to_numpy()
gain_phi = pattern["Gain_p(dB)"].to_numpy()
Antenna1 = Antenna(0.2,4,gain_theta,gain_phi)
#Antenna2 = Antenna(0.4,4,gain_theta,gain_phi)

#Points of the movement (for the moment it is straight)
p1 = np.array([0,0,0])
p2 = np.array([4,0,0])
#Initialization of robot with antennas
Robot_inst = Robot(p1,[Antenna1])
#p3 serves as a visual guide to know where the movement starts
kappa = 0.1
p3 = (1-kappa)*p1+kappa*p2


data_list = []  #list containing all the saved data
#tag position params
x_tag = 2 # Lets spice it up a bit
z_tag = 0.1
y_steps = np.linspace(0.1,3,1000) #the last number is the number of samples for each epoch
for y_tag in y_steps:
    #Tag initialization
    tagp = np.array([x_tag,y_tag,z_tag])
    tag1 = Tag(tagp)
    #do this for all antennas
    for antenna_num,antenna in enumerate(Robot_inst.antennas):
        phases = []
        robot_pos = []
        rho = 0   #rho is the position of the x axis
        mu,sigma = 0.0038,0.001834  #mu and sigma for the normal distribution of the sampling 
        while(rho<4):
            rho = rho + np.random.normal(mu,sigma)
            #there will be a problem with z if p1 and p2 do not have z = 0
            new_pos = p1 + rho*(p2-p1)/np.linalg.norm(p2-p1)
            robot_pos.append(new_pos)
            Robot_inst.update_pos(new_pos)
            r = np.linalg.norm(tag1.position-Robot_inst.position)
            phases.append( ((2*np.pi)/lambda_signal)*2*r + np.random.normal(0,0.09)) #mu = 0 and sigma = 5deg ~= 0.09 rad
        data_list.append({
            "x_tag": tag1.position[0],
            "y_tag": tag1.position[1],
            "z_tag": tag1.position[2],
            "Antenna": antenna_num,
            "robot_pos" : np.array(robot_pos),
            "Phases": np.array(phases)
        })

# Convert the list to a DataFrame
df = pd.DataFrame(data_list)

# Save to an Excel file
df.to_pickle("Sim_Data_Noisy.pkl")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   x_tag      1000 non-null   float64
 1   y_tag      1000 non-null   float64
 2   z_tag      1000 non-null   float64
 3   Antenna    1000 non-null   int64  
 4   robot_pos  1000 non-null   object 
 5   Phases     1000 non-null   object 
dtypes: float64(3), int64(1), object(2)
memory usage: 47.0+ KB


In [19]:
len(df.Phases[56]), len(df.Phases[75]) # Different values so an error occurs!

(1059, 1047)